# Day 3: Machine learning - Training Evaluation dan Prediction

## 1. XGBoost Training dan Accuracy Assessement

Seperti telah dijelaskan sebelumnya, pada hari kedua data sampel telah dibagi menjadi dua bagian, yaitu 80% untuk training dan 20% untuk testing. Seluruh data tersebut telah disimpan dalam format .npy, sehingga pada tahap ini kita hanya perlu memanggilnya kembali untuk proses pelatihan model dan evaluasi akurasi.

Pada sesi hari ketiga, peserta akan mempelajari penggunaan algoritma machine learning Extreme Gradient Boosting (XGBoost) dengan memanfaatkan library xgboost di Python. Library ini dirancang secara khusus untuk memudahkan pengguna dalam mengimplementasikan algoritma XGBoost yang dikenal efisien, fleksibel, dan portabel. Library xgboost mengimplementasikan algoritma pembelajaran mesin berbasis kerangka kerja Gradient Boosting.

In [ ]:
# Load training dan testing data yang sudah disimpan dalam format Numpy Array
import numpy as np

X_train = np.load('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/X_train.npy')
X_test = np.load('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/X_test.npy')
y_train = np.load('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/y_train.npy')
y_test = np.load('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/y_test.npy')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Import XGBoost classifier dari library xgboost
from xgboost import XGBClassifier

# Buat model XGBoost dengan default parameter
xgb_model_default = XGBClassifier(random_state=42)

Dokumentasi Parameter XGboost: https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
#Cek parameter Xgboost model
xgb_model_default.get_xgb_params()

In [ ]:
#Training XGboost Model
xgb_model_default.fit(X_train, y_train)

In [ ]:
# Evaluation assessment
# Menggunakan scikit-learn library untuk menghitung overall accuacy, classification report, dan confusion metrix
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Menerapkan xgb_model_default untuk memprediksi kelas dari tetsing dataset
y_pred_default = xgb_model_default.predict(X_test)

In [ ]:
# Membuat evaluation function untuk mempermudah kedepannya
def evaluate_classification(y_true, y_pred):
    # Perhitungan overall accuracy
    OA = accuracy_score(y_true, y_pred)

    # Print Overall Accuracy
    print(f"Overall Accuracy: {OA:.4f}")

    # Menghitung classification report: precision, recall, f1-score
    report = classification_report(y_true, y_pred, digits=4)
    print(report)

    # Menghitung confusion matrix
    class_names = ['Tubuh Air', 'Vegetasi Non-Mangrove', 'Mangrove', 'Lahan Terbangun', 'Lahan Terbuka']
    cm = confusion_matrix(y_true, y_pred)

    # Visualisasi confusion matrix menggunakan seaborn heatmap
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix', fontsize=14)
    plt.show()

In [ ]:
# Menerapkan function evaluation assessment dengan data yang sebenarnya atau y_test dan data hasil prediksi
evaluate_classification(y_test, y_pred_default)

## 2. Hyperprameter Tuning

Hyperparameter adalah “pengaturan dari luar” sebuah model machine learning. Nilainya tidak dipelajari otomatis dari data, tapi ditentukan oleh kita sebelum model dijalankan

jadi intinya Hyperparameter tuning adalah proses mencoba berbagai kombinasi pengaturan hyperparameter untuk mencari yang paling optimal.

Tujuannya supaya model bisa belajar dengan baik → akurasi tinggi, error kecil, dan tidak overfitting.

### 2.1 Melakukan Hyperparameter Tuning secara manual

Terdapat beebrapa parameter yang dapat kita definiskan untuk melakukan hyperparameter tuning. Beberapa paramater penting yang dapat kita definisikan adalah: n_estimator, max_depth, eta, gamma, subsample, dan colsample_bytree.

- n_estimators, number of trees atau jumlah pohon yang ada didalam model (default=100)
- max_depth, mengntrol maximum depth of tress pada model (default=6)
- eta, eta atau learning rate yang mengontrol step size dimana optimizer akan membuat updates kedalam nilai bobot/weights (default=0.3)
- gamma, atau min_split_loss adalah minimum loss reduction yang diperlukan untuk membuat partisi lebih lanjut pada leaf node of the tree (default=0)
- subsample, Parameter subsampel mengendalikan fraksi pengamatan yang digunakan untuk setiap pohon (default=1)
- colsample_bytree, Parameter colsample_bytree mengontrol fraksi fitur yang digunakan untuk setiap pohon (default=1)

In [ ]:
xgb_model_modify = XGBClassifier(random_state=42, n_estimators=50, max_depth=3, eta=0.3, gamma=0.5, subsample=0.5, colsample_bytree=0.5)
#Cek parameter Xgboost model
xgb_model_modify.get_xgb_params()

In [ ]:
#Training XGboost Model
xgb_model_modify.fit(X_train, y_train)

In [ ]:
# Evaluation assessment

# Menerapkan xgb_model_default untuk memprediksi kelas dari tetsing dataset
y_pred_modify = xgb_model_modify.predict(X_test)

# Menerapkan function evaluation assessment dengan data yang sebenarnya atau y_test dan data hasil prediksi
evaluate_classification(y_test, y_pred_modify)

Kita bisa melihat hasil dari parameter yang kita definisikan secara manual menghasilkan akurasi yang sedikit lebih jelek dari hasil model secara default.
Oleh karena itu, kita dapat menggunakan metode gridsearch untuk melakukan hyperparameter tuning secara otomatis

### 2.2 Hyperparameter tuning menggunakan GridSearchCV

untuk mempermudah, untuk tuning disini kita akan mencoba menggunakan GridSearchCV. GridSearchCV adalah metode di scikit-learn untuk melakukan pencarian hyperparameter terbaik dengan cara mencoba semua kombinasi yang mungkin dari daftar hyperparameter yang kita tentukan.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# XGBoost Classifier awal
init_xgb_model = XGBClassifier(random_state=42)

In [ ]:
# Mendeifiniskan list beberapa parameter yang akan digunakan untuk hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 300],
    'max_depth': [3, 6, 9],
    'eta': [0.1, 0.3, 0.5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

In [ ]:
# GridSearchCV dengan 5-fold cross-validation
grid_search = GridSearchCV(init_xgb_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=5)

In [ ]:
import time

# Start the timer
start_time = time.time()

# Train XGBoost model dengan grid_search
grid_search.fit(X_train, y_train)

# End the timer
end_time = time.time()

# Calculate and print the time taken
elapsed_time = end_time - start_time
print(f"Waktu yang dibutuhkan untuk running adalah: {elapsed_time:.3f} seconds")

bisa di lihat pada pemberitahuan output nya bahwa:

- 324 candidates → ada 324 kombinasi parameter berbeda yang didefinisikan di dalam param_grid. Jadi GridSearchCV akan mencoba melatih MLP dengan 324 konfigurasi berbeda.
- 5 folds → setiap kombinasi parameter diuji menggunakan 5-fold cross-validation. Jadi, untuk satu kombinasi parameter, model akan dilatih dan divalidasi 5 kali dengan pembagian data yang berbeda.
- totalling 1620 fits → total keseluruhan proses pelatihan (fit) yang dilakukan adalah 324 × 5 = 1620
Dengan kata lain, GridSearchCV sedang melakukan 1620 kali training dan evaluasi untuk mencari kombinasi parameter terbaik dengan hasil akurasi paling tinggi.

In [ ]:
# Print the best parameters
print("Best parameters adalah:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

In [ ]:
# Evaluation assessment

#Menerapkan best parameter xgb
best_xgb = grid_search.best_estimator_

# Menerapkan xgb_model_default untuk memprediksi kelas dari tetsing dataset
y_pred_grid = best_xgb.predict(X_test)

# Menerapkan function evaluation assessment dengan data yang sebenarnya atau y_test dan data hasil prediksi
evaluate_classification(y_test, y_pred_grid)

In [ ]:
#Membuat feature importance

from xgboost import plot_importance
import matplotlib.pyplot as plt

#Band List
band_description = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI', 'NDBI', 'MNDWI', 'NDMI', 'CMRI']

#set band list as feature name
best_xgb.get_booster().feature_names = band_description

#Plot feature importance
plt.figure(figsize=(10, 8))
plot_importance(best_xgb, importance_type='weight')
plt.title('Feature Importance in xgboost')
plt.show()

**Save Model**

In [ ]:
import joblib

# Save the model
joblib.dump(best_xgb, '/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/best_xgb_model.pkl')

## 3. Menghasilkan Peta Penutup Lahan

In [ ]:
!pip install rasterio

import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import matplotlib.pyplot as plt
import joblib

In [ ]:
# Open raster file (Sentinel-2 data)
raster = rasterio.open('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 1 & 2/Data Pelatihan Sesi 1 & 2/S2_with_SpectralIndices.tif')

# import AOi kita menggunakan geopandas
AOI = gpd.read_file('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 1 & 2/Data Pelatihan Sesi 1 & 2/AOI.shp')

In [ ]:
# Load saved model
loaded_model = joblib.load('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/best_xgb_model.pkl')

In [ ]:
# Apply model untuk mengkelasifikasikan keseluruhan citra
def classify_raster(raster, model):
    raster_data = raster.read().reshape((raster.count, -1)).T  # Reshape to (pixels, bands)
    mask = np.isnan(raster_data).any(axis=1)  # Exclude NaN pixels
    classified = np.full(raster_data.shape[0], np.nan)
    classified[~mask] = model.predict(raster_data[~mask])
    return classified.reshape((raster.height, raster.width))

In [ ]:
classified_map = classify_raster(raster, loaded_model)

# Day 4: Visualization

## 1. Visualisasi peta statis di python

**Visualisasi hasil klasifikasi**

In [ ]:
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

# Mendefinisikan Legend dan warna untuk setiap kelas
legend_labels = ['Tubuh Air', 'Vegetasi Non-Mangrove', 'Mangrove', 'Lahan Terbangun', 'Lahan Terbuka']
legend_colors = ['blue', 'green', 'orange', 'Red', 'Brown']

# Membuat figure untuk dua subplot: 1 untuk Sentinel-2 dan 2. untuk  hasil klasifikasi penutup lahan
fig, ax = plt.subplots(1, 2, figsize=(18, 8))

# Display Original Sentinel-2 image (NIR-SWIR1-RED)
rgb = np.stack([raster.read(4), raster.read(5), raster.read(3)], axis=-1)
brightness_factor = 3.5
S2_Vis_rgb = np.clip(rgb * brightness_factor, 0, 1)

ax[0].imshow(S2_Vis_rgb, extent=[AOI.total_bounds[0], AOI.total_bounds[2],AOI.total_bounds[1], AOI.total_bounds[3]])
ax[0].set_title('Sentinel-2 (NIR-SWIR1-RED)')

# Klasifikasi penutup lahan
# Mendefinisikan colormap
cmap = ListedColormap(legend_colors)

# Display hasil klasifikasi penutup lahan
im = ax[1].imshow(classified_map, cmap=cmap, interpolation='nearest', extent=[AOI.total_bounds[0], AOI.total_bounds[2],AOI.total_bounds[1], AOI.total_bounds[3]])
ax[1].set_title('Peta Klasifikasi Penutup Lahan')

# Membuat legend
legend_handles = [mpatches.Patch(color=legend_colors[i], label=legend_labels[i]) for i in range(len(legend_labels))]
# menambahkan legend kedalam plot
ax[1].legend(handles=legend_handles, loc='upper left', title="Land Cover Classes", fontsize='x-small', title_fontsize='medium', edgecolor="black", fancybox=True)

plt.tight_layout()
plt.show()


**Menyimpan hasil klasifikasi kedalam format raster .tif**

In [ ]:
import rasterio
from rasterio.transform import from_origin

def export_classified_map(classified_map, reference_raster, output_path):
    # Mendapatkan metadata dari data Sentinel-2
    meta = reference_raster.meta.copy()

    # Update the metadata
    meta.update({
        "driver": "GTiff",
        "dtype": 'int32',  # Untuk integer class (0,1,2,3,4)
        "count": 1,        # Hanya satu band
        "compress": "lzw"  # file compression
    })

    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(classified_map.astype('int32'), 1)

# Menerapkan fungsi yang telah dibuat
output_tif_path = '/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/classification_Map_xgboost.tif'
export_classified_map(classified_map, raster, output_tif_path)

Pada sesi ke empat ini, kita alebih berfokus bagaimana cara nya membuat sebuah visualisasi peta dengan menggunakan python dan juga QGIS.

pada python, kita akan mempelejari bagaimana caranya membuat visualisasi secara interaktif dengan menggunakan library leaflet yang sudah tersedia di python

## 2. Visualisasi Peta Interaktif dengan leaflet

Pada bagian ini, kita akan mempelajari cara membuat visualisasi peta interaktif menggunakan library Leafmap. Leafmap adalah library Python yang memudahkan visualisasi data spasial dan analisis dengan menyediakan kemampuan untuk menambahkan lapisan data, berinteraksi dengan peta, dan membuat peta tematik tanpa harus menulis banyak kode.

Sumber belajar/dokumentasi library Leafmap [**LINK**](https://leafmap.org/)

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!pip install leafmap
!pip install mapclassify

from leafmap import leafmap
import matplotlib.colors as mcolors

In [ ]:
LC = gpd.read_file('/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/LandCover_Bali_2024.shp')

In [ ]:
print(LC)

In [ ]:
# Mendapatkan bounds dari GeoDataFrame (AOI Teluk Benoa)
bounds = LC.to_crs('EPSG:4326').total_bounds  # [minx, miny, maxx, maxy]

In [ ]:
bounds

In [ ]:
# Menghitung center dari bounding box
center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

In [ ]:
center

In [ ]:
# Mendapatkan bounds dari GeoDataFrame (AOI Teluk Benoa)
bounds = LC.to_crs('EPSG:4326').total_bounds  # [minx, miny, maxx, maxy]

# Menghitung center dari bounding box
center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# Mapping nilai DN ke nama kategori
kategori_mapping = {
    0: "Badan Air",
    1: "Vegetasi Non Mangrove",
    2: "Mangrove",
    3: "Lahan Terbangun",
    4: "Lahan Terbuka"
}

# Buat kolom baru bernama 'Kategori'
LC['Kategori'] = LC['DN'].map(kategori_mapping)

# Define the list of colors
legend_colors = ['blue', 'red', 'brown', 'orange', 'green']

# Create a custom colormap from the list of colors
cmap = mcolors.ListedColormap(legend_colors)

# Create an interactive map
m = leafmap.Map(center=center, zoom=13)
m.add_data(
    LC,
    column="Kategori",  # Ganti dari "DN" ke "Kategori"
    cmap=cmap,  # Gunakan cmap untuk warna
    legend_title="Klasifikasi Tutupan Lahan",
    layer_name="Tutupan Lahan Teluk Benoa Bali",
    style={'fillOpacity': 0.7, 'weight': 0}  # weight: 0 menghilangkan garis tepi
)
m

**Membuat Peta time Series Interaktif**

In [ ]:
!pip install localtileserver

In [ ]:
m = leafmap.Map(center=center, zoom=13)
images = ['/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/classification_Map_2017.tif',
          '/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/classification_Map_2020.tif',
          '/content/drive/MyDrive/Geosoftware ID/Training - 1 November 2025/Data/Sesi 3 & 4/Data pelatihan Sesi 3 & 4/classification_Map_xgboost.tif'
          ]

m.add_time_slider(
    images,
    time_interval=0.5,
    position="bottomright",
    band=[1],
    zoom_to_layer=True,
    cmap="RdYlGn"
)
m